In [1]:
# step 0: spatially join business data to tazs using "contains" so businesses on taz boundaries don't get dropped

In [2]:
import numpy as np
import pandas as pd

In [3]:
taz_bus = pd.read_csv('./taz1454_BAbusinesses_contains.csv')
naics = pd.read_csv('./naics_recode.csv')

In [4]:
# drop one copy of businesses that were joined to two tazs because they are on the boundary
print(len(taz_bus))
taz_bus_null = taz_bus[taz_bus['JOIN_FID'] == -1]
taz_bus_notnull = taz_bus[taz_bus['JOIN_FID'] != -1]
taz_bus = taz_bus_notnull.drop_duplicates('JOIN_FID', keep='last')

323630


In [5]:
# join businesses and six categories
taz_bus['naicssix'] = taz_bus['NAICS'].astype(str).str[:6].astype(int)
taz_bus_null['naicssix'] = 0
taz_bus = taz_bus.append(taz_bus_null)
taz_bus_naics = pd.merge(taz_bus, naics, on='naicssix', how='left')
print(len(taz_bus_naics))

C:\Users\etheocharides\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


323626


In [6]:
# number of tazs
len(taz_bus_naics['TAZ1454'].unique())

1454

In [7]:
# create steelhead columns and populate them with no. of employees
for i in taz_bus_naics['sixcat'].unique():
    taz_bus_naics[i] = 0
    for j in np.arange(len(taz_bus_naics)):
        if taz_bus_naics['sixcat'][j] == i:
            taz_bus_naics.loc[j,i] = taz_bus_naics['EMPNUM'][j]
taz_bus_naics.describe()

,OBJECTID_x,Join_Count,TARGET_FID,JOIN_FID,SUPERD,TAZ1454,Shape__Are,Shape__Len,diff,LOCNUM,...,naics2,remi70,FPSEMPN,OTHEMPN,HEREEMPN,RETEMPN,MWTEMPN,MISSING,AGREMPN,nan
count,323626.0,323626.000000,323626.000000,323626.000000,323626.000000,323626.000000,323626.0,323626.0,323626.000000,3.236250e+05,...,323625.000000,323625.000000,323626.000000,323626.000000,323626.000000,323626.000000,323626.000000,323626.000000,323626.000000,323626.0
mean,-1.0,0.999997,767.678234,161919.133969,15.355481,739.755394,0.0,0.0,2503.641518,4.941072e+08,...,899.231048,154.407020,2.282576,1.610371,3.867220,1.545398,1.935682,0.088664,0.042565,0.0
std,0.0,0.001758,430.524526,93529.621065,9.514729,452.548579,0.0,0.0,5519.912071,2.146817e+08,...,1696.402438,863.828437,36.563921,34.725106,60.360497,18.178067,54.278985,2.702688,2.884014,0.0
min,-1.0,0.000000,0.000000,-1.000000,1.000000,1.000000,0.0,0.0,-2735.000000,1.629500e+04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,-1.0,1.000000,401.000000,80909.250000,8.000000,347.000000,0.0,0.0,178.000000,4.029439e+08,...,52.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,-1.0,1.000000,751.000000,161847.500000,15.000000,739.000000,0.0,0.0,688.000000,4.266076e+08,...,61.000000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,-1.0,1.000000,1168.000000,242948.750000,22.000000,1152.000000,0.0,0.0,2253.000000,6.447916e+08,...,81.000000,56.000000,1.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0
max,-1.0,1.000000,1453.000000,323903.000000,34.000000,1454.000000,0.0,0.0,55026.000000,9.989136e+08,...,4849.000000,6768.000000,10001.000000,8000.000000,17383.000000,4500.000000,11269.000000,1100.000000,900.000000,0.0


In [8]:
# clean up columns and summarize employment by taz
taz_bus_naics_col = taz_bus_naics[['TAZ1454', 'EMPNUM', 'AGREMPN', 'FPSEMPN', 'HEREEMPN', 'MWTEMPN', 'OTHEMPN', 'RETEMPN', 'MISSING']]
taz_bus_naics_sum = taz_bus_naics_col.groupby(['TAZ1454']).sum()
taz_bus_naics_sum['OTHEMPN'] = taz_bus_naics_sum['OTHEMPN'] + taz_bus_naics_sum['MISSING'] # fold missing into othempn
taz_bus_naics_sum = taz_bus_naics_sum.drop(['MISSING'], axis=1)
taz_bus_naics_sum.head()

,EMPNUM,AGREMPN,FPSEMPN,HEREEMPN,MWTEMPN,OTHEMPN,RETEMPN,MISSING
TAZ1454,,,,,,,,
1,16674.0,10.0,13069.0,1298.0,341.0,1307.0,649.0,364.0
2,25943.0,0.0,17533.0,3281.0,1023.0,1228.0,2878.0,173.0
3,2218.0,0.0,714.0,902.0,67.0,396.0,139.0,32.0
4,18250.0,1.0,11124.0,1754.0,1834.0,2705.0,832.0,228.0
5,18613.0,0.0,3523.0,6055.0,2658.0,555.0,5822.0,133.0


In [11]:
# check that employment totals are the same at the beginning and end
print(taz_bus['EMPNUM'].sum())
print((taz_bus_naics_sum['AGREMPN'].sum()+taz_bus_naics_sum['FPSEMPN'].sum()+taz_bus_naics_sum['HEREEMPN'].sum()
      +taz_bus_naics_sum['MWTEMPN'].sum()+taz_bus_naics_sum['OTHEMPN'].sum()+taz_bus_naics_sum['RETEMPN'].sum()))

3680429.0
3680429.0


In [13]:
# check that the correct no. of tazs come out
len(taz_bus_naics_sum)

1454

In [12]:
taz_bus_naics_sum.to_csv('./2015_employment_TAZ1454.csv')